In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

In [2]:
xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\new_data.csv', delimiter=',', dtype=np.float32)
x_data=xy[:,0:-1]
x_data = normalize(x_data, axis=0, norm='max')
y_data=xy[:,[-1]]
m_data=xy[:499,:]

F_data=xy[:,:]

In [14]:
# Model Inputs
def model_inputs(real_dim, noise_dim):
    inputs_real_ = tf.placeholder(tf.float32, shape=[None, real_dim], name='inputs_real')
    inputs_z_ = tf.placeholder(tf.float32, shape=[None, noise_dim], name='inputs_z')
    
    return inputs_real_, inputs_z_

#def leaky_relu(x, alpha):
 #   return tf.maximum(alpha * x, x)

In [15]:
# Generator Network# Genera 
def model_generator(z_input, out_dim, n_units=128, reuse=False, alpha=0.01):
    # used to reuse variables, name scope
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(z_input, n_units, activation=tf.nn.relu)
        logits = tf.layers.dense(hidden_layer, out_dim, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [16]:
# Discriminator Network
def model_discriminator(input, n_units=128, reuse=False, alpha=0.1):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(input, n_units, activation=tf.nn.relu)
        #hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, 1, activation=None
                                )
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [28]:
input_size = 42
z_dim = 21
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.1
smooth = 0.1
learning_rate = 0.001

In [29]:
tf.reset_default_graph()  # If we don't have this, as we call this block over and over, the graph gets bigger and bigger

graph = tf.Graph()
with graph.as_default():
    inputs_real, inputs_z = model_inputs(input_size, z_dim)
    
    g_outputs, g_logits = model_generator(inputs_z, input_size, n_units=g_hidden_size, reuse=False, alpha=alpha)
    
    d_outputs_real, d_logits_real = model_discriminator(inputs_real, n_units=d_hidden_size, reuse=False, alpha=alpha)
    d_outputs_fake, d_logits_fake = model_discriminator(g_outputs, n_units=d_hidden_size, reuse=True, alpha=alpha)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real) * (1-smooth)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))
    
    d_loss = d_loss_real + d_loss_fake
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))
    
    t_vars = tf.trainable_variables()
    g_vars = [variable for variable in t_vars if 'generator' in variable.name]
    d_vars = [variable for variable in t_vars if 'discriminator' in variable.name]
    
    # Affected Variables with var_list
    d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(d_loss, var_list=d_vars)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_loss, var_list=g_vars)
    
    # Saving variables with var_list
    saver = tf.train.Saver(var_list=g_vars)

In [53]:
samples=[]
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    f = open('C:\\Users\\SANHA\\Desktop\\gen_sample.txt', 'w')
    for step in range(10):
        
        
        batch_images = F_data[step].reshape([1, 42])
        batch_z = np.random.uniform(0, 1, size=[1, z_dim])
        
        _ = sess.run(d_optimizer, feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        _ = sess.run(g_optimizer, feed_dict={inputs_z : batch_z})
        loss_d, loss_g = sess.run([d_loss, g_loss], feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        if step%100==0:
            print('step {} / {} Complete. D_Loss : {:0.3f}, G_Loss : {:0.3f}'.format(step+1, 10000, loss_d, loss_g))
        
        sample_z = np.random.uniform(0, 1, size=[1, z_dim])  # 16 Samples each epoch
        gen_samples, _ = sess.run(model_generator(inputs_z, input_size, reuse=True), feed_dict={inputs_z : sample_z})
        gen_dd = gen_samples.resize(1, 42)
        
        f.write(str(gen_samples.reshape([1,42])))
        samples.append(gen_samples)
        print(samples)
    
    print('Training Complete. ')
    f.close()

step 1 / 10000 Complete. D_Loss : 2.738, G_Loss : 0.531
[array([[0.51512474, 0.5073992 , 0.52457553, 0.5155208 , 0.50677305,
        0.5487645 , 0.50034076, 0.5663439 , 0.41488406, 0.5643594 ,
        0.5703508 , 0.46931487, 0.55612403, 0.45238727, 0.5224685 ,
        0.48038408, 0.48368222, 0.516634  , 0.4035194 , 0.5091744 ,
        0.48928183, 0.5118968 , 0.52392465, 0.54226565, 0.3740786 ,
        0.5225658 , 0.56165284, 0.49789643, 0.43710965, 0.5239433 ,
        0.40594935, 0.48174012, 0.4989671 , 0.4064424 , 0.5045853 ,
        0.4152441 , 0.5170811 , 0.43971494, 0.53367394, 0.55845964,
        0.52403563, 0.46870258]], dtype=float32)]
[array([[0.51512474, 0.5073992 , 0.52457553, 0.5155208 , 0.50677305,
        0.5487645 , 0.50034076, 0.5663439 , 0.41488406, 0.5643594 ,
        0.5703508 , 0.46931487, 0.55612403, 0.45238727, 0.5224685 ,
        0.48038408, 0.48368222, 0.516634  , 0.4035194 , 0.5091744 ,
        0.48928183, 0.5118968 , 0.52392465, 0.54226565, 0.3740786 ,
        

[array([[0.51512474, 0.5073992 , 0.52457553, 0.5155208 , 0.50677305,
        0.5487645 , 0.50034076, 0.5663439 , 0.41488406, 0.5643594 ,
        0.5703508 , 0.46931487, 0.55612403, 0.45238727, 0.5224685 ,
        0.48038408, 0.48368222, 0.516634  , 0.4035194 , 0.5091744 ,
        0.48928183, 0.5118968 , 0.52392465, 0.54226565, 0.3740786 ,
        0.5225658 , 0.56165284, 0.49789643, 0.43710965, 0.5239433 ,
        0.40594935, 0.48174012, 0.4989671 , 0.4064424 , 0.5045853 ,
        0.4152441 , 0.5170811 , 0.43971494, 0.53367394, 0.55845964,
        0.52403563, 0.46870258]], dtype=float32), array([[0.5747138 , 0.4987482 , 0.5345112 , 0.50096285, 0.5121119 ,
        0.57965267, 0.56690806, 0.59317666, 0.459032  , 0.5203882 ,
        0.5814408 , 0.46348032, 0.533925  , 0.42925   , 0.48777273,
        0.595202  , 0.47827446, 0.5042383 , 0.47737765, 0.5252579 ,
        0.5148778 , 0.5561938 , 0.4611873 , 0.5035606 , 0.4456527 ,
        0.43135503, 0.5559479 , 0.55363   , 0.33043465, 0.4678399

[array([[0.51512474, 0.5073992 , 0.52457553, 0.5155208 , 0.50677305,
        0.5487645 , 0.50034076, 0.5663439 , 0.41488406, 0.5643594 ,
        0.5703508 , 0.46931487, 0.55612403, 0.45238727, 0.5224685 ,
        0.48038408, 0.48368222, 0.516634  , 0.4035194 , 0.5091744 ,
        0.48928183, 0.5118968 , 0.52392465, 0.54226565, 0.3740786 ,
        0.5225658 , 0.56165284, 0.49789643, 0.43710965, 0.5239433 ,
        0.40594935, 0.48174012, 0.4989671 , 0.4064424 , 0.5045853 ,
        0.4152441 , 0.5170811 , 0.43971494, 0.53367394, 0.55845964,
        0.52403563, 0.46870258]], dtype=float32), array([[0.5747138 , 0.4987482 , 0.5345112 , 0.50096285, 0.5121119 ,
        0.57965267, 0.56690806, 0.59317666, 0.459032  , 0.5203882 ,
        0.5814408 , 0.46348032, 0.533925  , 0.42925   , 0.48777273,
        0.595202  , 0.47827446, 0.5042383 , 0.47737765, 0.5252579 ,
        0.5148778 , 0.5561938 , 0.4611873 , 0.5035606 , 0.4456527 ,
        0.43135503, 0.5559479 , 0.55363   , 0.33043465, 0.4678399

In [219]:
m_data[12].ndim
samples[499].ndim
m_data=np.vstack((m_data, samples))

ValueError: all the input arrays must have same number of dimensions

In [45]:
#xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\test.csv', delimiter=',', dtype=np.float32)
#x_data=xy[:,0:-1]
#y_data=xy[:,[-1]]
#F_data=xy[:,:]


x1_data=samples[:,0:-1]
y1_data=samples[:,[-1]]



nb_classes=2

X=tf.placeholder(tf.float32,[None,41])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W1=tf.Variable(tf.random_normal([41,128]),name='weight1')
b1=tf.Variable(tf.random_normal([128]),name='bias1')
layer1=tf.sigmoid(tf.matmul(X,W1)+b1)

W2=tf.Variable(tf.random_normal([128,128]),name='weight2')
b2=tf.Variable(tf.random_normal([128]),name='bias2')
layer2=tf.sigmoid(tf.matmul(layer1,W2)+b2)

W3=tf.Variable(tf.random_normal([128,nb_classes]),name='weight3')
b3=tf.Variable(tf.random_normal([nb_classes]),name='bias3')
logits=tf.matmul(layer2,W3)+b3
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

prediction=tf.argmax(hypothesis,1) #가능성을 퍼센트로~~
correct_prediction=tf.equal(prediction,tf.arg_max(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(100):
        sess.run(optimizer,feed_dict={X:x_data,Y:y_data})
        #print("training by gan sample")
        if step %10==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={X:x1_data,Y:y1_data})
            print("step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
  
    tr=0
    fa=0
    total=0


    pred = sess.run(prediction, feed_dict={X: x1_data})
    for p, y in zip(pred, y1_data.flatten()):
            if(p==int(y)):
                tr=tr+1
            else:
                fa=fa+1
            print("[{}] Prediction: {} Real Y: {}".format(p == int(y), p, int(y)))

    print("true={} false: {} acc: {:0.2f}%".format(tr,fa,tr/(tr+fa)))

SyntaxError: invalid syntax (<ipython-input-45-0b3ec2e802f1>, line 7)

In [46]:
gen_samples

array([[2.4553970e-08, 7.1718619e-05, 9.9986875e-01, 1.0000000e+00,
        9.9988890e-01, 7.7457592e-08, 2.7783786e-08, 3.6493454e-08,
        1.1885601e-08, 5.0298588e-09, 7.5024200e-09, 2.0793270e-07,
        5.9557110e-09, 5.4148814e-09, 3.9429771e-09, 4.2589861e-09,
        4.4374295e-09, 3.8925773e-08, 1.7269945e-08, 2.2296133e-08,
        1.1007133e-08, 7.3512356e-09, 9.9999988e-01, 4.9759774e-04,
        2.7375846e-08, 1.3688073e-08, 2.0449331e-06, 9.9995494e-01,
        9.9999821e-01, 2.3811012e-08, 9.0502709e-08, 9.9999976e-01,
        9.9996257e-01, 9.4371615e-08, 3.4825018e-08, 1.2443605e-06,
        1.2389594e-05, 1.8276279e-07, 5.9335091e-04, 2.0709102e-07,
        1.1110276e-05, 9.7890549e-08]], dtype=float32)

In [47]:
samples

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,